In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
#lectura de el csv de la ruta de la linea 154
df_ruta_linea = pd.read_csv("../data/raw/ruta_linea154.csv")
#lectura de el excel de accidentalidad
df_accidentalidad = pd.read_excel("../data/raw/AccidentalidadBicicletas_2024.xlsx")
#lectua de el archivo xml
ruta_entradas= "../data/raw/2024Entradas.xml"
df_entradas = pd.read_xml(ruta_entradas)
#lectura de el json de las estaciones
ruta_estaciones = "../data/raw/estacionesBicicletas.json"
# abrimos el archivo json y lo convertimos a un DataFrame
with open(ruta_estaciones, 'r', encoding='utf-8') as f:
    data_json = json.load(f)
# convertimos el json a un DataFrame
df_estaciones = pd.json_normalize(data_json)



In [3]:
df_ruta_linea.head()

,linea_id,linea_nombre,orden,parada
0,154,MADRID (Chamartín) - S.S. REYES CIRCULAR (Por ...,1,EST.CHAMARTÍN
1,154,MADRID (Chamartín) - S.S. REYES CIRCULAR (Por ...,2,Pº CASTELLANA-MANUEL CALDEIRO
2,154,MADRID (Chamartín) - S.S. REYES CIRCULAR (Por ...,3,Pº CASTELLANA-VIEJAS
3,154,MADRID (Chamartín) - S.S. REYES CIRCULAR (Por ...,4,LLANO CASTELLANO-FCO.SANCHA
4,154,MADRID (Chamartín) - S.S. REYES CIRCULAR (Por ...,5,LLANO CASTELLANO-VALDEGOVIA


In [4]:
df_accidentalidad.head()

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga
0,2023S040280,2024-01-04,14:09:00,AVDA. NICETO ALCALA ZAMORA / AUTOV. M-11,3,16,HORTALEZA,Colisión fronto-lateral,Lluvia débil,Motocicleta > 125cc,Conductor,De 55 a 59 años,Hombre,2.0,Ingreso inferior o igual a 24 horas,444913.056,4481427.179,N,NaN
1,2023S040280,2024-01-04,14:09:00,AVDA. NICETO ALCALA ZAMORA / AUTOV. M-11,3,16,HORTALEZA,Colisión fronto-lateral,Lluvia débil,Turismo,Conductor,De 55 a 59 años,Mujer,14.0,Sin asistencia sanitaria,444913.056,4481427.179,N,NaN
2,2023S040309,2024-02-15,14:05:00,CALL. TESORO / CALL. MINAS,18,1,CENTRO,Colisión fronto-lateral,Lluvia débil,Bicicleta,Conductor,De 25 a 29 años,Hombre,7.0,Asistencia sanitaria sólo en el lugar del acci...,440122.814,4475169.548,N,NaN
3,2023S040309,2024-02-15,14:05:00,CALL. TESORO / CALL. MINAS,18,1,CENTRO,Colisión fronto-lateral,Lluvia débil,Motocicleta hasta 125cc,Conductor,De 35 a 39 años,Hombre,14.0,Sin asistencia sanitaria,440122.814,4475169.548,N,NaN
4,2023S040310,2024-02-18,10:40:00,GTA. RUIZ JIMENEZ / CALL. SAN BERNARDO,3,7,CHAMBERÍ,Colisión lateral,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,NaN,NaN,440136.981,4475721.316,N,NaN


In [5]:
df_entradas.head()

,idVehiculo,deposito,tipoVehiculo,motivo,turno,grua,fechaEntrada,tipoVia,via,numero,cp
0,CM20241216060430,COMISARIA,3.0,29,3,S387,2024-12-16T06:02:00,CALLE,CUESTA,1,28026
1,CO20240101041149,COLON,3.0,44,3,S364,2024-01-01T04:05:00,CALLE,MARTIN DE LOS HEROS,73,28008
2,CO20240101051114,COLON,3.0,46,3,S300,2024-01-01T05:10:00,CALLE,VITAL AZA,50,28017
3,CO20240101051804,COLON,3.0,44,3,S363,2024-01-01T05:10:00,CALLE,PRINCESA DE EBOLI,16,28050
4,CO20240101112324,COLON,3.0,7,1,S363,2024-01-01T11:22:00,CALLE,FUENCARRAL,126,28004


In [6]:
df_estaciones.head()

,$type,id,url,commonName,placeType,additionalProperties,children,childrenUrls,lat,lon
0,"Tfl.Api.Presentation.Entities.Place, Tfl.Api.P...",BikePoints_1,/Place/BikePoints_1,"River Street , Clerkenwell",BikePoint,[{'$type': 'Tfl.Api.Presentation.Entities.Addi...,[],[],51.529163,-0.109970
1,"Tfl.Api.Presentation.Entities.Place, Tfl.Api.P...",BikePoints_2,/Place/BikePoints_2,"Phillimore Gardens, Kensington",BikePoint,[{'$type': 'Tfl.Api.Presentation.Entities.Addi...,[],[],51.499606,-0.197574
2,"Tfl.Api.Presentation.Entities.Place, Tfl.Api.P...",BikePoints_3,/Place/BikePoints_3,"Christopher Street, Liverpool Street",BikePoint,[{'$type': 'Tfl.Api.Presentation.Entities.Addi...,[],[],51.521283,-0.084605
3,"Tfl.Api.Presentation.Entities.Place, Tfl.Api.P...",BikePoints_4,/Place/BikePoints_4,"St. Chad's Street, King's Cross",BikePoint,[{'$type': 'Tfl.Api.Presentation.Entities.Addi...,[],[],51.530059,-0.120973
4,"Tfl.Api.Presentation.Entities.Place, Tfl.Api.P...",BikePoints_5,/Place/BikePoints_5,"Sedding Street, Sloane Square",BikePoint,[{'$type': 'Tfl.Api.Presentation.Entities.Addi...,[],[],51.493130,-0.156876


In [7]:
def limpieza_general_df(df: pd.DataFrame, nombre_df: str = "DataFrame") -> pd.DataFrame:
    """
    Limpieza general del DataFrame sin eliminar duplicados.
    - Elimina columnas vacías
    - Normaliza nombres de columnas
    - Rellena NaN con 'desconocido'
    - Convierte texto a minúsculas y sin espacios
    - Intenta convertir a números o fechas sin usar argumentos deprecados
    """
    print(f"\nlimpieza de: {nombre_df}")
    df = df.copy()

    # 1. Eliminar columnas completamente vacías
    df.dropna(axis=1, how="all", inplace=True)

    # 2. Limpiar nombres de columnas
    df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns] # Normalizar nombres de columnas

    # 3. Limpieza de columnas
    for col in df.columns: # Iterar sobre cada columna
        df[col] = df[col].fillna("desconocido") # Rellenar NaN con 'desconocido'

        if df[col].dtype == object:
            df[col] = df[col].astype(str).str.strip().str.lower() # Convertir a string, eliminar espacios y pasar a minúsculas
            df[col] = df[col].replace("NaN", "desconocido")# Reemplazar 'nan' por 'desconocido'
        else:
            try:
                df[col] = pd.to_numeric(df[col]) # Intentar convertir a numérico
                df[col] = df[col].replace(np.nan, "desconocido") # Reemplazar NaN por 'desconocido'
            except Exception:
                pass

    print(f" Limpieza completada: {nombre_df}. {df.shape[0]} filas, {df.shape[1]} columnas.")
    return df

In [8]:
limpieza_general_df(df_ruta_linea)
limpieza_general_df(df_accidentalidad)
limpieza_general_df(df_entradas)
limpieza_general_df(df_estaciones)



limpieza de: DataFrame
 Limpieza completada: DataFrame. 42 filas, 4 columnas.

limpieza de: DataFrame
 Limpieza completada: DataFrame. 50416 filas, 19 columnas.

limpieza de: DataFrame
 Limpieza completada: DataFrame. 35283 filas, 11 columnas.

limpieza de: DataFrame
 Limpieza completada: DataFrame. 801 filas, 10 columnas.


,$type,id,url,commonname,placetype,additionalproperties,children,childrenurls,lat,lon
0,"tfl.api.presentation.entities.place, tfl.api.p...",bikepoints_1,/place/bikepoints_1,"river street , clerkenwell",bikepoint,[{'$type': 'tfl.api.presentation.entities.addi...,[],[],51.529163,-0.109970
1,"tfl.api.presentation.entities.place, tfl.api.p...",bikepoints_2,/place/bikepoints_2,"phillimore gardens, kensington",bikepoint,[{'$type': 'tfl.api.presentation.entities.addi...,[],[],51.499606,-0.197574
2,"tfl.api.presentation.entities.place, tfl.api.p...",bikepoints_3,/place/bikepoints_3,"christopher street, liverpool street",bikepoint,[{'$type': 'tfl.api.presentation.entities.addi...,[],[],51.521283,-0.084605
3,"tfl.api.presentation.entities.place, tfl.api.p...",bikepoints_4,/place/bikepoints_4,"st. chad's street, king's cross",bikepoint,[{'$type': 'tfl.api.presentation.entities.addi...,[],[],51.530059,-0.120973
4,"tfl.api.presentation.entities.place, tfl.api.p...",bikepoints_5,/place/bikepoints_5,"sedding street, sloane square",bikepoint,[{'$type': 'tfl.api.presentation.entities.addi...,[],[],51.493130,-0.156876
...,...,...,...,...,...,...,...,...,...,...
796,"tfl.api.presentation.entities.place, tfl.api.p...",bikepoints_274,/place/bikepoints_274,"warwick road, olympia",bikepoint,[{'$type': 'tfl.api.presentation.entities.addi...,[],[],51.496712,-0.205284
797,"tfl.api.presentation.entities.place, tfl.api.p...",bikepoints_768,/place/bikepoints_768,"clapham common north side, clapham common",bikepoint,[{'$type': 'tfl.api.presentation.entities.addi...,[],[],51.461343,-0.159322
798,"tfl.api.presentation.entities.place, tfl.api.p...",bikepoints_525,/place/bikepoints_525,"blythe road, olympia",bikepoint,[{'$type': 'tfl.api.presentation.entities.addi...,[],[],51.496927,-0.213500
799,"tfl.api.presentation.entities.place, tfl.api.p...",bikepoints_673,/place/bikepoints_673,"hibbert street, battersea",bikepoint,[{'$type': 'tfl.api.presentation.entities.addi...,[],[],51.465170,-0.180389


In [9]:
seleccion_columnas = df_estaciones[["$type", "url", "additionalProperties", "children", "childrenUrls"]]

In [10]:
df_estaciones.drop(columns=list(seleccion_columnas.columns), inplace=True)

In [11]:
df_ruta_linea.to_csv("../data/clean/df_ruta_linea_clean.csv", index=False)
df_accidentalidad.to_csv("../data/clean/df_accidentalidad_clean.csv", index=False)
df_entradas.to_csv("../data/clean/df_entradas_clean.csv", index=False)
df_estaciones.to_csv("../data/clean/df_estaciones_clean.csv", index=False)